In [1]:
import numpy as np
import pandas as pd
import os 
from itertools import product

import warnings

#from modshogun import *

from sklearn import linear_model, decomposition
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, GroupKFold, LeaveOneGroupOut
from sklearn.externals.joblib import Parallel, delayed
from sklearn.preprocessing import RobustScaler, LabelEncoder, StandardScaler, Imputer, MinMaxScaler
from sklearn.pipeline import Pipeline

from CustomCVs import KFoldMixedSizes, StratifiedKFoldMixedSizes, StratifiedKFoldByGroups
#from evaluation_classifier import Evaluater

from time import time
from IPython.display import clear_output

from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute, IterativeSVD #, MICE

from six.moves import cPickle as pickle

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
def create_dataset(df, columns, complete=False):
    if complete:
        tmp_df = df[~df.loc[:, np.append("Dx", columns)].T.isnull().any()]
    else:
        tmp_df = df[~df.loc[:, ["Dx"]].T.isnull().any()]

# remove subjects whom miss over 10% of their features

#         nans_per_subject = tmp_df.loc[:, tmp_df.columns.values[:]].isnull().T.sum()
#         completeness_per_subject = (float(tmp_df.shape[1]) - nans_per_subject) / tmp_df.shape[1]
#         tmp_df = tmp_df.loc[completeness_per_subject >= 0.9]
        
    X = tmp_df.loc[:, columns].as_matrix()
    y = tmp_df.loc[:, "Dx"].values.astype(int)

    tmp_df['age_group_tesla_site'] = tmp_df["age_group"] + tmp_df["tesla"] + tmp_df["site"]
    le = LabelEncoder()
    groups = le.fit_transform(tmp_df.age_group_tesla_site.values)
    
    return X, y, groups

In [3]:
data_dir="/data/rmthomas/HeteroSmallSample"
ENIGMA_OCD_df = pd.read_csv(os.path.join(data_dir, "ENIGMA_OCD.csv"))
ENIGMA_OCD_df.head()

# Specify columns of interest for covariates and FS features
covariates_columns = [ENIGMA_OCD_df.columns.values[3], ENIGMA_OCD_df.columns.values[6]]
print(covariates_columns)
subcort_columns = ENIGMA_OCD_df.columns.values[9:9+16]
print(subcort_columns[0], subcort_columns[-1])
cort_s_columns = ENIGMA_OCD_df.columns.values[9+16:25+70]
print(cort_s_columns[0], cort_s_columns[-1])
cort_t_columns = ENIGMA_OCD_df.columns.values[25+70:25+70+70]
print(cort_t_columns[0], cort_t_columns[-1])

print(ENIGMA_OCD_df.shape)
print(ENIGMA_OCD_df[~ENIGMA_OCD_df.T.isnull().any()].shape)
print(ENIGMA_OCD_df[~ENIGMA_OCD_df.loc[:, covariates_columns].T.isnull().any()].shape)
print(ENIGMA_OCD_df[~ENIGMA_OCD_df.loc[:, subcort_columns].T.isnull().any()].shape)
print(ENIGMA_OCD_df[~ENIGMA_OCD_df.loc[:, cort_s_columns].T.isnull().any()].shape)
print(ENIGMA_OCD_df[~ENIGMA_OCD_df.loc[:, cort_t_columns].T.isnull().any()].shape)

ENIGMA_OCD_df['age_group_tesla_site'] = ENIGMA_OCD_df["age_group"] + ENIGMA_OCD_df["tesla"] + ENIGMA_OCD_df["site"]

columns_to_choose = list(range(3, len(ENIGMA_OCD_df.columns) - 1))
columns_to_choose.pop(1)
columns_to_choose.pop(1)

columns_to_choose = ENIGMA_OCD_df.columns.values[columns_to_choose]

X_tot, y_tot, groups_tot = create_dataset(ENIGMA_OCD_df, columns_to_choose, complete=False)
X_c_tot, y_c_tot, groups_c_tot = create_dataset(ENIGMA_OCD_df, columns_to_choose, complete=True)

subcort_idx = np.where(columns_to_choose == "LLatVent")[0][0]
cort_surf_idx = np.where(columns_to_choose == "LSurfArea")[0][0]
cort_thick_idx = np.where(columns_to_choose == "LThickness")[0][0]

['site', 'AgeSQ']
Clean Laccumb
Lamyg R_parsorbitalis_surfavg
R_parstriangularis_surfavg R_parsorbitalis_thickavg
(4370, 180)
(683, 180)
(4243, 180)
(1450, 180)
(2114, 180)
(2409, 180)


/data/local/softwares/anaconda3/envs/py36ml/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]
/data/local/softwares/anaconda3/envs/py36ml/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [ ]:
columns_to_choose = list(range(3, len(ENIGMA_OCD_df.columns) - 1))
print(len(columns_to_choose))


In [ ]:
# Calculate NaNs per column (features)
nans_per_column = np.isnan(X_tot).sum(axis=0)

# Count NaNs per subjects and select only those who have at least one NaN 
nans_per_subject = np.isnan(X_tot).sum(axis=1)[np.isnan(X_tot).sum(axis=1) > 0]
n_to_pick_from = len(nans_per_subject)

# Now from these "has-NaN" subjects select the N equals the total "has-no-NaN" subjects
n_to_choose = X_tot[~np.isnan(X_tot).any(axis=1), :].shape[0]

# We pick only those "has-NaN" subjects with the highest amount of NaNs
sorted_nans_per_subject = nans_per_subject[np.argsort(nans_per_subject)]
indices = np.round(np.linspace(0, n_to_pick_from-1, num=n_to_choose))

nans_per_subject = sorted_nans_per_subject[np.array(indices, dtype=int)]
# nans_per_subject[0] = 0

# Here we calculate the NaN probabilites per column and subject
p_columns = nans_per_column / float(nans_per_column.sum())
p_subjects = nans_per_subject / float(nans_per_subject.sum())

# Calculate percentage of missing data in original data set
missing_data_percentage =  float(nans_per_column.sum()) /  X_tot.size

# Obtain a complete dataset without NaNs that we can use for imputation simulations
X_to_impute = X_c_tot.copy()

# Create a flat array with matrix indices used for mapping
items = np.arange(X_to_impute.size)

# Calculate probability matrix for each element
prob_matrix = np.reshape(p_columns, (1, p_columns.shape[0])) * np.reshape(p_subjects, (p_subjects.shape[0], 1))

In [ ]:
(prob_matrix > 0).sum() / float(prob_matrix.size)

In [ ]:
print p_subjects.max()
print p_columns.max()

In [ ]:
true_perc_nan_col = (nans_per_column / float(nans_per_column.sum())) * 100

true_perc_nan_subc = true_perc_nan_col[:cort_surf_idx].sum()
true_perc_nan_cort_s = true_perc_nan_col[cort_surf_idx:cort_thick_idx].sum()
true_perc_nan_cort_t = true_perc_nan_col[cort_thick_idx:].sum()

print true_perc_nan_subc
print true_perc_nan_cort_s
print true_perc_nan_cort_t

In [ ]:
X_with_nans[~np.isnan(X_with_nans).any(axis=1), :]

In [ ]:
impute_regression(X_with_nans, X_with_nans)

In [ ]:
def impute_regression(X_train, X_test):
    
    # Check if training data contains any columns which only contain NaNs 
    X_train_NaN_only_column_idx = np.where(np.isnan(X_train).sum(axis=0) == X_train.shape[0])[0]
    
    if len(X_train_NaN_only_column_idx) > 0:    
        X_train_imputed = np.delete(X_train, X_train_NaN_only_column_idx, 1)
        X_test_imputed = np.delete(X_test, X_train_NaN_only_column_idx, 1)
    else:
        X_train_imputed, X_test_imputed = X_train.copy(), X_test.copy()

    n_subjects, n_features = X_train_imputed.shape
    X_train_no_nans = X_train_imputed[~np.isnan(X_train_imputed).any(axis=1), :]
    
    if X_train_no_nans.shape[0] <= 1:
        return None
    
    corr_matrix = np.corrcoef(X_train_no_nans.T)

    r, c = np.where(np.isnan(X_test))

    rows_with_nan, rows_index, rows_counts = np.unique(r, return_index=True, return_counts=True)

    for row, idx, counts in zip(rows_with_nan, rows_index, rows_counts):

        missing_columns = c[idx:idx+counts]

        corr_rows = corr_matrix[missing_columns]

        # Set correlation to itselve and other missing features to zero 
        corr_rows[:, missing_columns] = 0

        # Take absolute from correlations and sort row ascendingly
        abs_corr_rows = np.abs(corr_rows)
        sorted_corr_indices = np.argsort(abs_corr_rows)

        n_corrs = n_features - len(missing_columns)

        n_t = 8
        n_c = 5

        if n_corrs < n_c:
            n_c = n_corrs
            n_t = n_corrs
        elif n_corrs < n_t:
            n_t = n_corrs

        for column_index in range(len(missing_columns)):
            
            missing_column = missing_columns[column_index] 

            # Randomly pick 5 out of 8 features with highest correlations
            chosen_indices = np.random.choice(sorted_corr_indices[column_index, -n_t:], n_c, replace=False)

            # Perform GLM on these 5 features to predict missing feature using the complete (no NaN) dataset
            X = X_train_no_nans[:, chosen_indices]
            y = X_train_no_nans[:, missing_column]

            reg = linear_model.LinearRegression()
            reg.fit(X, y)

            X_ = X_test[row, chosen_indices].reshape(1, -1)
            y_ = reg.predict(X_)

            X_test_imputed[row, missing_column] = np.squeeze(y_)

    return X_test_imputed

In [ ]:
def run_imputation(X_with_nans_test, strategy):
    
    if strategy=="mean":
        mean_imputer = Imputer(missing_values='NaN', strategy="mean", axis=0)
        X_imputed = mean_imputer.fit_transform(X_without_nans_train)
        X_test_imputed = mean_imputer.transform(X_with_nans_test)
        
    if strategy=="median":
        median_imputer = Imputer(missing_values='NaN', strategy="median", axis=0)
        X_imputed = median_imputer.fit_transform(X_without_nans_train)
        X_test_imputed = median_imputer.transform(X_with_nans_test)
    
    if strategy=="regression":
        X_test_imputed = impute_regression(X_without_nans_train, X_with_nans_test)
        
    if strategy=="knn":
        
        complete_id = range(X_with_nans_test.shape[0] + X_without_nans_train.shape[0])
        train_id = list(set(complete_id) - set(test_id))
        
        X_to_impute = np.zeros((len(complete_id), X_without_nans_train.shape[1]))
        X_to_impute[train_id] = X_without_nans_train
        X_to_impute[test_id] = X_with_nans_test
        
        X_imputed = KNN(k=5, verbose=False).complete(X_to_impute)
        X_test_imputed = X_imputed[test_id]
    
    return X_imputed

In [ ]:
# with open('simulated_nans_data.pickle', 'rb') as handle:
#     bool_nan_ind_container = pickle.load(handle)

In [ ]:
# bool_nan_ind_0 = bool_nan_ind_container[-1, -1]
# bool_nan_ind_0.sum()

In [ ]:
strategies = ['mean', 'median', 'regression', 'knn']
n_splits = 10

skfg = StratifiedKFoldByGroups(n_splits=n_splits) 

i = 0
total_iterations = len(strategies) * fracs.size * n_resample * n_splits

masked_MAE_scores = np.zeros((len(strategies), fracs.size, n_resample, n_splits))
masked_MSE_scores = np.zeros((len(strategies), fracs.size, n_resample, n_splits))
masked_RMSE_scores = np.zeros((len(strategies), fracs.size, n_resample, n_splits))

masked_NMAE_scores = np.zeros((len(strategies), fracs.size, n_resample, n_splits))
masked_NMSE_scores = np.zeros((len(strategies), fracs.size, n_resample, n_splits))
masked_NRMSE_scores = np.zeros((len(strategies), fracs.size, n_resample, n_splits))

MAE_scores = np.zeros((len(strategies), fracs.size, n_resample, n_splits))
MSE_scores = np.zeros((len(strategies), fracs.size, n_resample, n_splits))
RMSE_scores = np.zeros((len(strategies), fracs.size, n_resample, n_splits))

NMAE_scores = np.zeros((len(strategies), fracs.size, n_resample, n_splits))
NMSE_scores = np.zeros((len(strategies), fracs.size, n_resample, n_splits))
NRMSE_scores = np.zeros((len(strategies), fracs.size, n_resample, n_splits))

for i_f in range(len(fracs)):

    # Create array to keep track of the amount of NaNs per column over resampling iterations
    nan_distribution_per_f = np.zeros((X_true.shape[1]))
    
    t1 = time()
    
    for i_n in range(n_resample):
        
        bool_nan_ind = bool_nan_ind_container[i_f, i_n]
        
        X_sim_nans = X_true.copy()
        X_sim_nans[bool_nan_ind] = np.nan
        
        nans_per_row = bool_nan_ind.sum(axis=1)
        
        nan_distribution_per_f = nan_distribution_per_f + bool_nan_ind.sum(axis=0)
        
        # Apply KFold (stratified for amount of NaNs per subject) to fit imputer on training data and assess it's performance on test data with NaN
        for id_iter_cv, (train_id, test_id) in enumerate(skfg.split(X_sim_nans, np.zeros_like(nans_per_row), nans_per_row)):
                    
            X_sim_nans_test = X_sim_nans[test_id]
            X_true_train, X_true_test =  X_true[train_id], X_true[test_id]
            
            bool_nan_ind_test = bool_nan_ind[test_id]
                
            for id_strategy, strategy in enumerate(strategies):
                
                # TODO: Pass X true 
                X_mean_imp_test = run_imputation(X_true_train, X_sim_nans_test, strategy, test_id)
                    
                # Mask all non imputed values with NaNs
                X_true_test_masked, X_mean_imp_test_masked = X_true_test.copy(), X_mean_imp_test.copy()
                X_true_test_masked[~bool_nan_ind_test], X_mean_imp_test_masked[~bool_nan_ind_test] = np.nan, np.nan
                
                # Suppress the warnings in this block
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore", category=RuntimeWarning)

                    MAE_per_column = np.nanmean(np.absolute(X_true_test_masked  - X_mean_imp_test_masked), axis=0)
                    MSE_per_column = np.nanmean(((X_true_test_masked - X_mean_imp_test_masked) ** 2), axis=0) 
                    RMSE_per_column = np.sqrt(MSE_per_column)  

                    MAE_masked = np.nanmean(MAE_per_column) 
                    MSE_masked = np.nanmean(MSE_per_column) 
                    RMSE_masked = np.nanmean(RMSE_per_column) 

                    # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4895707/
                    NMAE_masked = np.nanmean(MAE_per_column / np.std(X_true_train, axis=0)) 
                    NMSE_masked = np.nanmean(MSE_per_column / np.std(X_true_train, axis=0)) 
                    NRMSE_masked = np.nanmean(RMSE_per_column / np.std(X_true_train, axis=0)) 

                    MAE = np.mean(np.mean(np.absolute(X_mean_imp_test - X_true_test), axis=0))
                    MSE = np.mean(np.mean((X_mean_imp_test - X_true_test)**2, axis=0))
                    RMSE = np.mean(np.sqrt(np.mean((X_mean_imp_test - X_true_test)**2, axis=0)))

                    NMAE = np.mean(np.mean(np.absolute(X_mean_imp_test - X_true_test), axis=0) / np.std(X_true_train, axis=0))
                    NMSE = np.mean(np.mean((X_mean_imp_test - X_true_test)**2, axis=0) / np.std(X_true_train, axis=0))
                    NRMSE = np.mean(np.sqrt(np.mean((X_mean_imp_test - X_true_test)**2, axis=0)) / np.std(X_true_train, axis=0))

                masked_MAE_scores[id_strategy, i_f, i_n, id_iter_cv] = MAE_masked
                masked_MSE_scores[id_strategy, i_f, i_n, id_iter_cv] = MSE_masked
                masked_RMSE_scores[id_strategy, i_f, i_n, id_iter_cv] = RMSE_masked
                
                masked_NMAE_scores[id_strategy, i_f, i_n, id_iter_cv] = NMAE_masked
                masked_NMSE_scores[id_strategy, i_f, i_n, id_iter_cv] = NMSE_masked
                masked_NRMSE_scores[id_strategy, i_f, i_n, id_iter_cv] = NRMSE_masked
                
                MAE_scores[id_strategy, i_f, i_n, id_iter_cv] = MAE
                MSE_scores[id_strategy, i_f, i_n, id_iter_cv] = MSE
                RMSE_scores[id_strategy, i_f, i_n, id_iter_cv] = RMSE
                
                NMAE_scores[id_strategy, i_f, i_n, id_iter_cv] = NMAE
                NMSE_scores[id_strategy, i_f, i_n, id_iter_cv] = NMSE
                NRMSE_scores[id_strategy, i_f, i_n, id_iter_cv] = NRMSE

                i += 1
    t2 = time()
    
    clear_output(wait=True)
    print("Finished calculating similarity between original and imputed data for {}% missing data \n Iteration: ({}/{}) completed in {} s \n".format(fracs[i_f]*100, i, total_iterations, t2 - t1))
    perc_nan_col = (nan_distribution_per_f / float(nan_distribution_per_f.sum())) * 100    
    print("{} % of induced NaNs in subcortical features, true percentage  = {} %".format(perc_nan_col[:cort_surf_idx].sum(), true_perc_nan_subc))
    print("{} % of induced NaNs in cortical surface area features, true percentage  = {} %".format(perc_nan_col[cort_surf_idx:cort_thick_idx].sum(), true_perc_nan_cort_s)) 
    print("{} % of induced NaNs in cortical thickness features, true percentage  = {} %".format(perc_nan_col[cort_thick_idx:].sum(), true_perc_nan_cort_t))

In [ ]:
files = [masked_MAE_scores, masked_MSE_scores, masked_RMSE_scores, 
         masked_NMAE_scores, masked_NMSE_scores, masked_NRMSE_scores,
         MAE_scores, MSE_scores, RMSE_scores,
         NMAE_scores, NMSE_scores, NRMSE_scores]

filenames = ['masked_MAE_scores', 'masked_MSE_scores', 'masked_RMSE_scores', 
             'masked_NMAE_scores', 'masked_NMSE_scores', 'masked_NRMSE_scores',
             'MAE_scores', 'MSE_scores', 'RMSE_scores',
             'NMAE_scores', 'NMSE_scores', 'NRMSE_scores']

for f, fname in zip(files, filenames):
    with open((fname + '.pickle'), 'wb') as handle:
        pickle.dump(f, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# strategies = ['mean', 'median', 'regression', 'knn']
# n_splits = 10

# masked_MAE_scores = np.zeros((len(strategies), fracs.size, n_resample, n_splits))
# masked_MSE_scores = np.zeros((len(strategies), fracs.size, n_resample, n_splits))
# masked_RMSE_scores = np.zeros((len(strategies), fracs.size, n_resample, n_splits))

# masked_NMAE_scores = np.zeros((len(strategies), fracs.size, n_resample, n_splits))
# masked_NMSE_scores = np.zeros((len(strategies), fracs.size, n_resample, n_splits))
# masked_NRMSE_scores = np.zeros((len(strategies), fracs.size, n_resample, n_splits))

# MAE_scores = np.zeros((len(strategies), fracs.size, n_resample, n_splits))
# MSE_scores = np.zeros((len(strategies), fracs.size, n_resample, n_splits))
# RMSE_scores = np.zeros((len(strategies), fracs.size, n_resample, n_splits))

# NMAE_scores = np.zeros((len(strategies), fracs.size, n_resample, n_splits))
# NMSE_scores = np.zeros((len(strategies), fracs.size, n_resample, n_splits))
# NRMSE_scores = np.zeros((len(strategies), fracs.size, n_resample, n_splits))


# files = [masked_MAE_scores, masked_MSE_scores, masked_RMSE_scores, 
#          masked_NMAE_scores, masked_NMSE_scores, masked_NRMSE_scores,
#          MAE_scores, MSE_scores, RMSE_scores,
#          NMAE_scores, NMSE_scores, NRMSE_scores]

# filenames = ['masked_MAE_scores', 'masked_MSE_scores', 'masked_RMSE_scores', 
#              'masked_NMAE_scores', 'masked_NMSE_scores', 'masked_NRMSE_scores',
#              'MAE_scores', 'MSE_scores', 'RMSE_scores',
#              'NMAE_scores', 'NMSE_scores', 'NRMSE_scores']

# for i, (f, fname) in enumerate(zip(files, filenames)):
#     with open(fname +'.pickle', 'rb') as handle:
#         data = pickle.load(handle)
#         files[i] = data
#         print("Finished loading {}".format(fname))
        
        
# masked_MAE_scores, masked_MSE_scores, masked_RMSE_scores, masked_NMAE_scores, masked_NMSE_scores, masked_NRMSE_scores, MAE_scores, MSE_scores, RMSE_scores, NMAE_scores, NMSE_scores, NRMSE_scores = files[0],  files[1],  files[2],  files[3],  files[4],  files[5],  files[6],  files[7],  files[8],  files[9],  files[10],  files[11]

In [ ]:
color_palette = ['r', 'b', 'g', 'y']

data_mean = np.mean(masked_NRMSE_scores, axis=3).mean(axis=2)
data_sd = np.mean(masked_NRMSE_scores, axis=3).std(axis=2)

y_min = np.min(data_mean - data_sd)
y_max = np.max(data_mean + data_sd)

plt.figure(figsize=(15, 10))
plt.ylabel('masked NRMSE scores', size='x-large')
plt.ylim([y_min, y_max])

for i, strategy in enumerate(strategies):

    plt.plot(fracs, data_mean[i, :], color_palette[i], label=strategy)

    plt.axvline(x=missing_data_percentage, color='black', ls='dashed', alpha=0.5)
    plt.fill_between(fracs, data_mean[i, :] - data_sd[i, :], data_mean[i, :] + data_sd[i, :], color=color_palette[i], alpha=0.1)
    
ticks = np.arange(0, max(fracs) + 0.05, 0.05)
labels = [str(int(tick * 100)) for tick in ticks]

plt.xticks(ticks, labels, rotation=45)
plt.xlabel('Missing data %', size='x-large')
plt.legend()

plt.show()

In [ ]:
color_palette = ['r', 'b', 'g', 'y']

data_mean = np.nanmean(NRMSE_scores, axis=3).mean(axis=2)
data_sd = np.nanmean(NRMSE_scores, axis=3).std(axis=2)

y_min = np.nanmin(data_mean - data_sd)
y_max = np.nanmax(data_mean + data_sd)

plt.figure(figsize=(15, 10))
plt.ylabel('NRMSE scores', size='x-large')
plt.ylim([y_min, y_max])

for i, strategy in enumerate(strategies):

    plt.plot(fracs, data_mean[i, :], color_palette[i], label=strategy, alpha=0.9)

    plt.axvline(x=missing_data_percentage, color='black', ls='dashed', alpha=0.5)
    plt.fill_between(fracs, data_mean[i, :] - data_sd[i, :], data_mean[i, :] + data_sd[i, :], color=color_palette[i], alpha=0.1)
    
ticks = np.arange(0, max(fracs) + 0.05, 0.05)
labels = [str(int(tick * 100)) for tick in ticks]

plt.xticks(ticks, labels, rotation=45)
plt.xlabel('Missing data %', size='x-large')
plt.legend()

plt.show()

In [ ]:
color_palette = ['r', 'b', 'g', 'y']

data_mean = np.nanmean(masked_MAE_scores, axis=3).mean(axis=2)
data_sd = np.nanmean(masked_MAE_scores, axis=3).std(axis=2)

y_min = np.nanmin(data_mean - data_sd)
y_max = np.nanmax(data_mean + data_sd)

plt.figure(figsize=(15, 10))
plt.ylabel('masked MAE scores', size='x-large')
plt.ylim([y_min, y_max])

for i, strategy in enumerate(strategies):

    plt.plot(fracs, data_mean[i, :], color_palette[i], label=strategy)

    plt.axvline(x=missing_data_percentage, color='black', ls='dashed', alpha=0.5)
    plt.fill_between(fracs, data_mean[i, :] - data_sd[i, :], data_mean[i, :] + data_sd[i, :], color=color_palette[i], alpha=0.1)

ticks = np.arange(0, max(fracs) + 0.05, 0.05)
labels = [str(int(tick * 100)) for tick in ticks]

plt.xticks(ticks, labels, rotation=45)
plt.xlabel('Missing data %', size='x-large')
plt.legend()

plt.show()

In [ ]:
color_palette = ['r', 'b', 'g', 'y']

data_mean = np.nanmean(MAE_scores, axis=3).mean(axis=2)
data_sd = np.nanmean(MAE_scores, axis=3).std(axis=2)

y_min = np.nanmin(data_mean - data_sd)
y_max = np.nanmax(data_mean + data_sd)

plt.figure(figsize=(15, 10))
plt.ylabel('MAE scores', size='x-large')
plt.ylim([y_min, y_max])

for i, strategy in enumerate(strategies):

    plt.plot(fracs, data_mean[i, :], color_palette[i], label=strategy)

    plt.axvline(x=missing_data_percentage, color='black', ls='dashed', alpha=0.5)
    plt.fill_between(fracs, data_mean[i, :] - data_sd[i, :], data_mean[i, :] + data_sd[i, :], color=color_palette[i], alpha=0.1)

    
ticks = np.arange(0, max(fracs) + 0.05, 0.05)
labels = [str(int(tick * 100)) for tick in ticks]

plt.xticks(ticks, labels, rotation=45)
plt.xlabel('Missing data %', size='x-large')
plt.legend()

plt.show()

In [ ]:
color_palette = ['r', 'b', 'g', 'y']

data_mean = np.nanmean(masked_MAE_scores, axis=3).mean(axis=2)
data_sd = np.nanmean(masked_MAE_scores, axis=3).std(axis=2)

y_min = np.nanmin(data_mean - data_sd)
y_max = np.nanmax(data_mean + data_sd)

plt.figure(figsize=(15, 10))
plt.ylabel('NMAE scores', size='x-large')
plt.ylim([y_min, y_max])

for i, strategy in enumerate(strategies):

    plt.plot(fracs, data_mean[i, :], color_palette[i], label=strategy)

    plt.axvline(x=missing_data_percentage, color='black', ls='dashed', alpha=0.5)
    plt.fill_between(fracs, data_mean[i, :] - data_sd[i, :], data_mean[i, :] + data_sd[i, :], color=color_palette[i], alpha=0.1)

    
ticks = np.arange(0, max(fracs) + 0.05, 0.05)
labels = [str(int(tick * 100)) for tick in ticks]

plt.xticks(ticks, labels, rotation=45)
plt.xlabel('Missing data %', size='x-large')
plt.legend()

plt.show()

In [ ]:
color_palette = ['r', 'b', 'g', 'y']

data_mean = np.nanmean(masked_NMSE_scores, axis=3).mean(axis=2)
data_sd = np.nanmean(masked_NMSE_scores, axis=3).std(axis=2)

y_min = np.nanmin(data_mean - data_sd)
y_max = np.nanmax(data_mean + data_sd)

plt.figure(figsize=(15, 10))
plt.ylabel('MSE scores', size='x-large')
plt.ylim([y_min, y_max])

for i, strategy in enumerate(strategies):
    
    

    plt.plot(fracs, data_mean[i, :], color_palette[i], label=strategy)

    plt.axvline(x=missing_data_percentage, color='black', ls='dashed', alpha=0.5)
    plt.fill_between(fracs, data_mean[i, :] - data_sd[i, :], data_mean[i, :] + data_sd[i, :], color=color_palette[i], alpha=0.1)

ticks = np.arange(0, max(fracs) + 0.05, 0.05)
labels = [str(int(tick * 100)) for tick in ticks]

plt.xticks(ticks, labels, rotation=45)
plt.xlabel('Missing data %', size='x-large')
plt.legend()

plt.show()

In [ ]:
color_palette = ['r', 'b', 'g', 'y']

data_mean = np.nanmean(NMSE_scores, axis=3).mean(axis=2)
data_sd = np.nanmean(NMSE_scores, axis=3).std(axis=2)

y_min = np.nanmin(data_mean - data_sd)
y_max = np.nanmax(data_mean + data_sd)

plt.figure(figsize=(15, 10))
plt.ylabel('MSE scores', size='x-large')
plt.ylim([y_min, y_max])

for i, strategy in enumerate(strategies):

    plt.plot(fracs, data_mean[i, :], color_palette[i], label=strategy)

    plt.axvline(x=missing_data_percentage, color='black', ls='dashed', alpha=0.5)
    plt.fill_between(fracs, data_mean[i, :] - data_sd[i, :], data_mean[i, :] + data_sd[i, :], color=color_palette[i], alpha=0.1)

ticks = np.arange(0, max(fracs) + 0.05, 0.05)
labels = [str(int(tick * 100)) for tick in ticks]

plt.xticks(ticks, labels, rotation=45)
plt.xlabel('Missing data %', size='x-large')
plt.legend()

plt.show()

In [ ]:
missing_data_percentage * 100